In [ ]:
get_ipython().run_cell_magic('capture', 'cont', '!pip install db-dtypes\n')

In [ ]:
cont.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
get_ipython().run_line_magic('matplotlib', 'inline')

In [ ]:
import os
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics

In [ ]:
import statsmodels.api as sm
import itertools

CONFIGURE THE BIGQUERY SETTINGS

In [ ]:
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [ ]:
#obtaining the unemployment data
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
"""

In [ ]:
# QUERY THE DATA ONCE
query_job = bigquery_client.query(query)
data = query_job.to_dataframe()
#data['date']= pd.to_datetime(data['date'])
data.head()

In [ ]:
#obtaining the submission file
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.prediction_list`
"""

In [ ]:
# QUERY THE DATA ONCE
query_job = bigquery_client.query(query)
submit = query_job.to_dataframe()
#data['date']= pd.to_datetime(data['date'])
submit.head()

In [ ]:
data.info()   #Feature Matrix
#X = data.drop("date",1)
#y = data["wind_speed"]          #Target Variable

In [ ]:
submit.info()

dropping duplicate rows in the data

In [ ]:
data.drop_duplicates(inplace=True, ignore_index=True)
data.info()

unique number of ID in the data

In [ ]:
data.uu_id.unique().size

replace all na values with 0

In [ ]:
data.fillna(0, inplace=True)

In [ ]:
data.info()

number of weeks in the data for a random ID

In [ ]:
test=data[data['uu_id']=='bbcb018f0e5e49e13636f6e78ce9f60f']
len(test)

convert the timeperiod column to datetime

In [ ]:
data['timeperiod']= pd.to_datetime(data['timeperiod'], format='%Y%m%d')

In [ ]:
plt.figure(figsize=(12,10))
cor = data.corr()
sns.heatmap(cor,  cmap=plt.cm.Reds)
plt.show()

In [ ]:
#Correlation with output variable
cor_target = abs(cor["total_claims"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.2]
print(relevant_features)

In [ ]:
columns_rel = relevant_features.index.to_list()

In [ ]:
data.plot(subplots=True, figsize=(20,24))

In [ ]:
min(data['timeperiod']),max(data['timeperiod'])

In [ ]:
data.dtypes

data = data.set_index('timeperiod')

In [ ]:
data.index

data['Year'] = data.index.year
data['Month'] = data.index.month
data['Day'] = data.index.day
Display a random sampling of 5 rows
data.sample(5, random_state=0)

In [ ]:
sns.set(rc={'figure.figsize':(11, 4)})

In [ ]:
data['total_claims'].plot(linewidth=.5);

In [ ]:
cols_plot = ['week_number','countyfips','tract','total_claims','edu_8th_or_less','edu_grades_9_11','edu_hs_grad_equiv','edu_post_hs',
            'edu_unknown', 'gender_female', 'gender_male', 'gender_na', 'race_amerindian', 'race_asian', 'race_black', 'race_noanswer', 
            'race_hawaiiannative', 'race_other', 'race_white']
axes = data[cols_plot].plot(marker='.', alpha=0.5, linestyle='None', figsize=(30, 25), subplots=True)
for ax in axes:
    ax.set_ylabel('features')

import matplotlib.dates as mdates
fig, ax = plt.subplots()
ax.plot(data.loc['2019-08':'2019-12', 'precipitation_data'], marker='o', linestyle='-')
ax.set_ylabel('Precipitation')
ax.set_title('Aug 2019-2020 Precipiation Data')

In [ ]:
#fig, axes = plt.subplots(3, 1, figsize=(11, 10), sharex=True)
#for name, ax in zip(['precipitation_data', 'min_temperature', 'max_temperature'], axes):
#    sns.boxplot(data=data, x='Month', y=name, ax=ax)
#    ax.set_ylabel('precipitation')
#    ax.set_title(name)
    # Remove the automatic x-axis label from all but the bottom subplot
#    if ax != axes[-1]:
#        ax.set_xlabel('')

sns.boxplot(data=data, x='Month', y='total_claims');

from statsmodels.graphics.tsaplots import plot_acf
plot_acf(x=data['max_temperature'], lags=50)

In [ ]:
columns_rel.append('timeperiod')
columns_rel.append('uu_id')
columns_rel

In [ ]:
data

In [ ]:
data_arima=data[columns_rel]
data_arima

In [ ]:
get_ipython().run_cell_magic('capture', '', "unique_id=list(data_arima['uu_id'].unique())\ndata_dict = {}\n\nfor i in unique_id:\n    j = data_arima[data_arima['uu_id']==i].groupby('timeperiod')['total_claims'].sum().reset_index()\n    j = j.set_index('timeperiod')\n    data_dict[i] = j\n")

data_arima = data_arima.groupby('uu_id')[['total_claims', 'timeperiod','uu_id']]
data_arima.head()

data_arima_2 = data_arima.set_index('timeperiod')
data_arima.index

In [ ]:
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
with io.capture_output() as captured:

In [ ]:
import csv

In [ ]:
filename ="param.csv"

In [ ]:
# opening the file using "with"
# statement
with open(filename, 'r') as data:
    for line in csv.DictReader(data):
        print(line)

In [ ]:
data

In [ ]:
csv.DictReader(data)

In [ ]:
import csv

In [ ]:
filename =pd.read_csv("param.csv")

opening the file using "with"
statement
with open(filename, 'r') as data:
   for line in csv.DictReader(data):
       print(line)

In [ ]:
filename.head()

In [ ]:
filename.to_dict()

In [ ]:
filename.head()

In [ ]:
filename.to_dict()

In [ ]:
filename.to_dict(list)

In [ ]:
filename.to_dict('list')

In [ ]:
param_dict=file_param.to_dict('list')
seasonal_dict=file_seasonal.to_dict('list')

In [ ]:
import csv

In [ ]:
file_param =pd.read_csv("param.csv")
file_seasonal =pd.read_csv("seasonal.csv")
# opening the file using "with"
# statement
#with open(filename, 'r') as data:
#    for line in csv.DictReader(data):
#        print(line)

In [ ]:
param_dict=file_param.to_dict('list')
seasonal_dict=file_seasonal.to_dict('list')

In [ ]:
len(data_dict)

In [ ]:
hundred=list(data_dict.keys())[:100]
two_hundred=list(data_dict.keys())[100:200]
three_hundred=list(data_dict.keys())[200:300]
four_hundred=list(data_dict.keys())[300:400]
five_hundred=list(data_dict.keys())[500:573]

In [ ]:
#results = {}
for key in hundred:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=tuple(param_dict[key]),
                                seasonal_order=tuple(seasonal_dict[key]),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
results = {}
for key in hundred:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=tuple(param_dict[key]),
                                seasonal_order=tuple(seasonal_dict[key]),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
len(results)

In [ ]:
#results = {}
for key in two_hundred:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=tuple(param_dict[key]),
                                seasonal_order=tuple(seasonal_dict[key]),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
len(results)

In [ ]:
#results = {}
for key in three_hundred:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=tuple(param_dict[key]),
                                seasonal_order=tuple(seasonal_dict[key]),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
len(results)

In [ ]:
#results = {}
for key in four_hundred:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=tuple(param_dict[key]),
                                seasonal_order=tuple(seasonal_dict[key]),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
len(results)

In [ ]:
#results = {}
for key in five_hundred:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=tuple(param_dict[key]),
                                seasonal_order=tuple(seasonal_dict[key]),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
len(results)

In [ ]:
results = {}
for key in hundred:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=tuple(param_dict[key]),
                                seasonal_order=tuple(seasonal_dict[key]),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
len(results)

In [ ]:
results = {}
for key in two_hundred:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=tuple(param_dict[key]),
                                seasonal_order=tuple(seasonal_dict[key]),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
len(results)

In [ ]:
results = {}
for key in hundred:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=tuple(param_dict[key]),
                                seasonal_order=tuple(seasonal_dict[key]),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
len(results)

In [ ]:
#results = {}
for key in two_hundred:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=tuple(param_dict[key]),
                                seasonal_order=tuple(seasonal_dict[key]),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
len(results)

In [ ]:
#results = {}
for key in three_hundred:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=tuple(param_dict[key]),
                                seasonal_order=tuple(seasonal_dict[key]),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
len(results)

In [ ]:
#results = {}
for key in four_hundred:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=tuple(param_dict[key]),
                                seasonal_order=tuple(seasonal_dict[key]),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
hundred=list(data_dict.keys())[:100]
two_hundred=list(data_dict.keys())[100:200]
three_hundred=list(data_dict.keys())[200:300]
four_hundred=list(data_dict.keys())[300:400]
five_hundred=list(data_dict.keys())[400:573]

In [ ]:
len(results)

In [ ]:
hundred=list(data_dict.keys())[:100]
two_hundred=list(data_dict.keys())[100:200]
three_hundred=list(data_dict.keys())[200:300]
four_hundred=list(data_dict.keys())[300:400]
five_hundred=list(data_dict.keys())[400:500]
six_hundred=list(data_dict.keys())[500:573]

In [ ]:
#results = {}
for key in five_hundred:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=tuple(param_dict[key]),
                                seasonal_order=tuple(seasonal_dict[key]),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
len(results)

In [ ]:
#results = {}
for key in six_hundred:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=tuple(param_dict[key]),
                                seasonal_order=tuple(seasonal_dict[key]),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
len(results)

In [ ]:
list(results.keys())

In [ ]:
x=list(results.keys())[0]

In [ ]:
results[x].plot_diagnostics(figsize=(16, 8))
plt.show()

In [ ]:
x=list(results.keys())[2]

In [ ]:
results[x].plot_diagnostics(figsize=(16, 8))
plt.show()

In [ ]:
data_dict[x]['total_claims']

In [ ]:
pred = results[x].get_prediction(start=pd.to_datetime('2022-09-26'), dynamic=False)

In [ ]:
data_dict[x]['total_claims']

In [ ]:
pred = results[x].get_prediction(start=pd.to_datetime('2022-09-10'), dynamic=False)

In [ ]:
x=list(results.keys())[1]

In [ ]:
pred = results[x].get_prediction(start=pd.to_datetime('2022-09-10'), dynamic=False)

In [ ]:
pred = results[x].get_prediction( dynamic=False)

In [ ]:
x=list(results.keys())[2]

In [ ]:
pred = results[x].get_prediction( dynamic=False)

In [ ]:
pred_ci = pred.conf_int()
ax = data_dict[x]['total_claims'].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7, figsize=(14, 7))
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)
ax.set_xlabel('Date')
ax.set_ylabel('Furniture Sales')
plt.legend()
plt.show()

In [ ]:
pred = results[x].get_prediction()

In [ ]:
pred_ci = pred.conf_int()
ax = data_dict[x]['total_claims'].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7, figsize=(14, 7))
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)
ax.set_xlabel('Date')
ax.set_ylabel('Furniture Sales')
plt.legend()
plt.show()

In [ ]:
pred = results[x].get_prediction(start=pd.to_datetime('2022-09-26'), dynamic=False)

In [ ]:
data_dict[x]['total_claims']

In [ ]:
pred = results[x].get_prediction(start=pd.to_datetime('2022-09-10'), dynamic=False)

In [ ]:
pred_ci = pred.conf_int()
ax = data_dict[x]['total_claims'].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7, figsize=(14, 7))
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)
ax.set_xlabel('Date')
ax.set_ylabel('Furniture Sales')
plt.legend()
plt.show()

In [ ]:
pred = results[x].get_prediction(start=pd.to_datetime('2022-09-10'), end=pd.to_datetime('2022-10-10'), dynamic=False)

In [ ]:
pred = results[x].get_prediction(start=pd.to_datetime('2022-09-10'), dynamic=False)

In [ ]:
pred_ci = pred.conf_int()
ax = data_dict[x]['total_claims'].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7, figsize=(14, 7))
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)
ax.set_xlabel('Date')
ax.set_ylabel('Furniture Sales')
plt.legend()
plt.show()

In [ ]:
pred = results[x].forecast()

In [ ]:
pred_ci = pred.conf_int()
ax = data_dict[x]['total_claims'].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7, figsize=(14, 7))
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)
ax.set_xlabel('Date')
ax.set_ylabel('Furniture Sales')
plt.legend()
plt.show()

In [ ]:
pred

In [ ]:
pred = results[x].get_prediction(start=pd.to_datetime('2022-09-10'), dynamic=False)

In [ ]:
pred_ci = pred.conf_int()
ax = data_dict[x]['total_claims'].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7, figsize=(14, 7))
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)
ax.set_xlabel('Date')
ax.set_ylabel('Furniture Sales')
plt.legend()
plt.show()

In [ ]:
pred = results[x].forecast()

In [ ]:
pred

In [ ]:
pred = []
for key in hundred:
        i=results[key].forecast()
        pred.append(i)
    #print(results.summary().tables[1])

In [ ]:
pred

In [ ]:
pred = results[x].forecast()

In [ ]:
pred.values

In [ ]:
pred = []
for key in hundred:
        i=results[key].forecast()
        pred.append(i.values)
    #print(results.summary().tables[1])

In [ ]:
pred

In [ ]:
pred = results[x].forecast()

In [ ]:
pred.values[0]

In [ ]:
pred = []
for key in hundred:
        i=results[key].forecast()
        pred.append(i.values[0])
    #print(results.summary().tables[1])

In [ ]:
pred

In [ ]:
len(pred)

In [ ]:
#pred = []
for key in two_hundred:
        i=results[key].forecast()
        pred.append(i.values[0])
    #print(results.summary().tables[1])

In [ ]:
len(pred)

In [ ]:
#pred = []
for key in three_hundred:
        i=results[key].forecast()
        pred.append(i.values[0])
    #print(results.summary().tables[1])

In [ ]:
len(pred)

In [ ]:
#pred = []
for key in four_hundred:
        i=results[key].forecast()
        pred.append(i.values[0])
    #print(results.summary().tables[1])

In [ ]:
len(pred)

In [ ]:
#pred = []
for key in five_hundred:
        i=results[key].forecast()
        pred.append(i.values[0])
    #print(results.summary().tables[1])

In [ ]:
len(pred)

In [ ]:
#pred = []
for key in six_hundred:
        i=results[key].forecast()
        pred.append(i.values[0])
    #print(results.summary().tables[1])

In [ ]:
len(pred)

In [ ]:
pred

In [ ]:
pred

In [ ]:
hundred+two_hundred

In [ ]:
uu_id=hundred+two_hundred+three_hundred+four_hundred+five_hundred+six_hundred

In [ ]:
len(uu_id)

In [ ]:
submit.head()

In [ ]:
first_trial = pd.DataFrame(uu_id, columns=['uu_id'])
first_trial.head()

In [ ]:
first_trial['total_claims']=pred

In [ ]:
first_trial.head()

In [ ]:
submit.head()

In [ ]:
pd.merge(submit, first_trial, on='uu_id', how='outer')

In [ ]:
final_file=pd.merge(submit, first_trial, on='uu_id', how='outer')

In [ ]:
final_file=pd.merge(submit, first_trial, on='uu_id', how='outer')
final_file.to_csv(./'submission.csv', index=False)

In [ ]:
final_file=pd.merge(submit, first_trial, on='uu_id', how='outer')
final_file.to_csv('./submission.csv', index=False)